In [15]:
# 外部ライブラリをインポート
import boto3
import os
import tiktoken
import openai
import pandas as pd
import time
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain import hub
from dotenv import load_dotenv
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_aws import ChatBedrock
from langchain_community.retrievers.bedrock import AmazonKnowledgeBasesRetriever

In [16]:
# 環境変数読み込み
load_dotenv(verbose=True)
dotenv_path = os.path.join("./.env")
load_dotenv(dotenv_path)
openai.api_key = os.getenv("OPENAI_API_KEY")
cohere_api_key = os.getenv("COHERE_API_KEY")

In [17]:
# データフォルダのパス
novel_file_path = "../data/validation-test/"

In [18]:
# OpenAI LLMの設定 (RAGの生成部分)
llm = ChatOpenAI(
    model_name="gpt-4o",
    openai_api_key=openai.api_key
)

In [19]:
# Bedrock Agent API呼び出し用のクライアントを作成する
agent = boto3.client(service_name='bedrock-agent-runtime', region_name='us-east-1')

# Bedrock Agent APIに渡す必要のある変数を事前に定義する
model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
model_arn = f'arn:aws:bedrock:us-east-1::foundation-model/{model_id}'
kb_id = "" #ここにKnowkedge BaseのIDを入れる


In [20]:

# 検索手段を指定
bedrock_retriever = AmazonKnowledgeBasesRetriever(
  knowledge_base_id=kb_id, #ナレッジベースIDを指定
  region_name="us-east-1",
  # クエリ分割
  orchestrationConfiguration={
    'queryTransformationConfiguration': {
        'type': 'QUERY_DECOMPOSITION'
    }
  },
  # 検索時の設定
  retrieval_config={
    "vectorSearchConfiguration": {
      "numberOfResults": 10,
      "overrideSearchType": "HYBRID",
      
      }
    }
  ) #ここで指定

In [21]:
template = """
あなたは親切で知識豊富なチャットアシスタントです。
<excerpts>タグには、ユーザーが知りたい情報に関連する複数のドキュメントの抜粋が含まれています。

<excerpts>{context}</excerpts>

これらの情報をもとに、<question>タグ内のユーザーの質問に対する回答を提供してください。

<question>{question}</question>

また、質問への回答は以下の点に留意してください:

- <excerpts>タグの内容を参考にするが、回答に<excerpts>タグを含めないこと。
- 簡潔に3つ以内のセンテンスで回答すること。
- 日本語で回答すること。
- 数量で答える問題の回答には単位を付けること.
- 質問に対して<excerpts>タグ内にある情報で、質問に答えるための情報がない場合は「分かりません」と答えること.
- 質問自体に誤りがあると判断される場合は「質問誤り」のみ答えること.

"""

In [22]:
# プロンプトのテンプレートを定義
prompt = ChatPromptTemplate.from_template(template)

# LLMを指定
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs={"max_tokens": 2048},
    region_name="us-east-1"
)

In [23]:
# リランカー
cohere_reranker = CohereRerank(
    top_n=3,# Rerankで3個取得
    cohere_api_key=cohere_api_key,
    model="rerank-multilingual-v3.0"
)

# ContextualCompressionRetrieverの準備
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_reranker,
    base_retriever=bedrock_retriever
)

In [25]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [29]:
# チェーンを定義（検索 → プロンプト作成 → LLM呼び出し → 結果を取得）

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | model
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": compression_retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

#chain = ({"context": compression_retriever, "question": RunnablePassthrough()} | prompt | model | StrOutputParser())

query = "" #ここにAIへの質問を入れる

#result = rag_chain_with_source.invoke(query)

In [27]:
# 提供されたCSVファイルを読み込み
query_df = pd.read_csv("../data/query.csv", encoding="utf-8")

In [ ]:
# 各問題に対して回答と証拠を取得する関数
def get_answer_and_evidence(problem):
    # RAGによる検索・回答生成
    result = rag_chain_with_source.invoke(problem)
    answer = result["answer"]  # 回答部分
    evidence = result["context"][0].page_content # 証拠部分を抽出
    return answer, evidence

# 各行の問題に対して処理を実行し、回答と証拠を取得
answers = []
evidences = []

for _, row in query_df.iterrows():
    # 問題文を取得
    problem = row['problem']
    print(f"start {problem}")
    # 各問題に対する回答と証拠を取得
    answer, evidence = get_answer_and_evidence(problem)
    answers.append(answer)
    evidences.append(evidence)
    time.sleep(3)

In [ ]:
for _, row in query_df.iterrows():
    # 問題文を取得
    problem = row['problem']
    if int(_) < 53:
        continue
    print(f"start {_}: {problem}")
    # 各問題に対する回答と証拠を取得
    answer, evidence = get_answer_and_evidence(problem)
    answers.append(answer)
    evidences.append(evidence)
    time.sleep(2)

In [43]:
# DataFrameに回答と証拠を追加
query_df['answer'] = answers
query_df['full_evidence'] = evidences

In [45]:
# 結果をCSVファイルとして保存
query_df.to_csv("../result/validation/output_with_answers_and_evidence_202401001_bedrock_full.csv", index=False)

In [46]:

# 回答内容を整形し、冗長な文章を削除する
def create_reanswer(question, answer):
    reanswer_template = """
    あなたはプロの編集者です。
    以下に質問文に対する回答文があります。
    質問文に対して回答文の中から最も簡潔に重要な内容のみを抽出してください。
    単語のみを回答しても構いません。

    # 質問文
    {question}	

    # 回答文
    {answer}
    """

    custom_reanswer_prompt = PromptTemplate.from_template(reanswer_template)

    reanswer_chain = custom_reanswer_prompt | llm | StrOutputParser()

    return reanswer_chain.invoke({"question": question, "answer": answer})

In [47]:
reanswers = []

for _, row in query_df.iterrows():
    # 問題文と回答を取得
    problem = row['problem']
    answer = row['answer']
    # 各問題に対する回答と証拠を取得
    reanswer = create_reanswer(problem, answer)
    reanswers.append(reanswer)

# DataFrameに整形済み回答を追加
query_df['answer'] = reanswers

In [49]:
# 結果をCSVファイルとして保存
query_df.to_csv("../result/validation/output_with_answers_and_evidence_202401001_bedrock.csv", index=False)

In [50]:
# LLMを使ってanswerに基づき、evidenceから200文字程度を抜き出す関数
def extract_relevant_evidence(answer, full_evidence):
    extract_prompt = PromptTemplate(
        input_variables=["answer", "full_evidence"],
        template=
            """
                f"以下は回答と関連する証拠文です。"
                f"回答に必要な部分を200文字以内で抜き出してください。\n"
                f"回答: {answer}\n\n"
                f"証拠文: {full_evidence}\n"
            """
    )
    chain = extract_prompt | llm

    response = chain.invoke(
        {"answer": answer, "full_evidence": full_evidence}
    )
    return response.content

In [51]:
# ステップ2: full_evidenceを使って関連する部分を抜き出す
query_df['evidence'] = query_df.apply(
    lambda row: extract_relevant_evidence(row['answer'], row['full_evidence']),
    axis=1
)

In [53]:
# 欠損値の確認
query_df['evidence'].isna().sum()

0

In [54]:
replace_dict = {
        "\n": "",
        "\r": "",
    }
query_df = query_df.replace(
        {"answer": replace_dict},
        regex=True
    )
query_df = query_df.replace(
        {"evidence": replace_dict},
        regex=True
    )

In [55]:
# LLMを使って要約を行う関数
def summarize_answer(answer: str) -> str:
    # OpenAI API などの LLM を使用して要約を実行
    
    summarize_prompt = PromptTemplate(
        input_variables=["answer"],
        template=
            """
                以下の文章を50文字程度で要約してください。\n
                f"回答: {answer}"
            """
    )
    chain = summarize_prompt | llm

    response = chain.invoke(
        {"answer": answer}
    )
    return response.content

In [56]:
# tiktokenとgpt-4のトークナイザーを取得
enc = tiktoken.encoding_for_model("gpt-4-2024-08-06")

# query_df の "answer" 列のトークン数を計算し、50トークンを超える場合は要約を行う関数
def check_and_summarize_answers(query_df: pd.DataFrame) -> pd.DataFrame:
    def summarize_if_needed(answer: str) -> str:
        # トークン数を計算
        token_count = len(enc.encode(answer))
        
        # トークン数が50を超えた場合は要約する
        if token_count > 50:
            # LLMを使って要約
            summarized_answer = summarize_answer(answer)
            return summarized_answer
        return answer

    # "answer" 列に対して処理を適用
    query_df["answer"] = query_df["answer"].apply(summarize_if_needed)
    return query_df

In [57]:
query_df = check_and_summarize_answers(query_df)

In [59]:
# 必要な列（id, answer, evidence）をヘッダなしでCSVに書き出し
query_df[['index', 'answer', 'evidence']].to_csv(
    "../data/evaluation/submit/predictions.csv",
    index=False,
    header=False,
    encoding="utf-8-sig"
)

In [60]:
# バックアップ用に別のファイルにも保存
query_df[['index', 'answer', 'evidence']].to_csv(
    "../data/evaluation/submit_example/validation/predictions_20241001_prompt_remove_stopworkd.csv",
    index=False,
    header=False,
    encoding="utf-8-sig"
)